Задание:
1. Выберите текстовую коллекцию, которая вм симптична (можно взять отсюда - http://romip.ru/ru/collections/index.html ).
2. Поставьте библиотеку https://github.com/seatgeek/fuzzywuzzy (самый простой вариант).
3. Выделите нечеткие дубликаты документов.
4. Сдайте мне лабу, надо обозначить 
длина шингла; 
процент дубликатов от общего количества документов;
выделенные кластеры из нечетких дубликатов (желательно с названиями по смыслу).

Кластеры можно визуализировать (и подписать легенду).


Как сдавать задание:
1. Код и визуализаци онтологии в любом виде должны лежать на Яндекс.диске. Лучше не видео, видео не влезет.
1.2. Можно прислать ссылку на личный гит с отдельной папкой "Поисковые системы/Лабораторная № 4".
2. У кода должны быть комменты, у картинки - легенда (хотя бы с расшифровкой связей).

In [19]:
from lxml import objectify
import pandas as pd

xml_data = objectify.parse('news-080404.xml')  # Parse XML data
root = xml_data.getroot()  # Root element

data = []
cols = []
for i in range(len(root.getchildren())):
    child = root.getchildren()[i]
    data.append([subchild.text for subchild in child.getchildren()])
    cols = [subchild.tag for subchild in child.getchildren()]

df = pd.DataFrame(data)  # Create DataFrame and transpose it
df.columns = cols  # Update column names
df

In [ ]:
import base64

def decode_base64(text):
    return base64.b64decode(text).decode('windows-1251')

df['text'] = df['content'].apply(decode_base64)
df['text']

0        Министр финансов РФ Алексей Кудрин предложил п...
1        Министерство природных ресурсов распорядилось ...
2        В четверг неподалеку от Эль-Фаллуджи иракские ...
3        Загорать можно и зимой - для этого достаточно ...
4        На вопросы наших читательниц о гнойных воспале...
                               ...                        
10015    Премьер-министр Пакистана Зафарулла Хан Джамал...
10016    Пакистанские полицейские предотвратили покушен...
10017    КАРАЧИ, 2 апреля.\nПолиции Пакистана удалось п...
10018    Боевик палестинской экстремистской группировки...
10019    11 сентября 2001 года, в день совершения терак...
Name: text, Length: 10020, dtype: object

In [ ]:
import string
import regex

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('russian'))

def to_clear_tokens(text):
    text = regex.sub('[\p{Latin}]', u'', text)
    text = regex.sub('(\s\d+)', '', text)
    text = regex.sub('``', '', text)
    word_tokens = word_tokenize(text)
    clear_lower_tokens = [w.lower() for w in word_tokens if (not w.lower() in stop_words and not w in string.punctuation)]
    return ' '.join(clear_lower_tokens)

df['text'] = df['text'].apply(to_clear_tokens)
df['text']

0        министр финансов рф алексей кудрин предложил п...
1        министерство природных ресурсов распорядилось ...
2        четверг неподалеку эль-фаллуджи иракские повст...
3        загорать зимой достаточно пару неделю ходить с...
4        вопросы наших читательниц гнойных воспалениях ...
                               ...                        
10015    премьер-министр пакистана зафарулла хан джамал...
10016    пакистанские полицейские предотвратили покушен...
10017    карачи апреля полиции пакистана удалось предот...
10018    боевик палестинской экстремистской группировки...
10019    11 сентября года день совершения терактов нью-...
Name: text, Length: 10020, dtype: object

In [ ]:
def form_shingles(text:str):
    shingles_count = 5

    words = text.split(' ')

    shingles = []
    for i in range(len(words) - shingles_count + 1):
        shingle = words[i:i+shingles_count]
        shingles.append(' '.join(shingle))

    return shingles

df['shingles'] = df['text'].apply(form_shingles)
df['shingles']

0        [министр финансов рф алексей кудрин, финансов ...
1        [министерство природных ресурсов распорядилось...
2        [четверг неподалеку эль-фаллуджи иракские повс...
3        [загорать зимой достаточно пару неделю, зимой ...
4        [вопросы наших читательниц гнойных воспалениях...
                               ...                        
10015    [премьер-министр пакистана зафарулла хан джама...
10016    [пакистанские полицейские предотвратили покуше...
10017    [карачи апреля полиции пакистана удалось, апре...
10018    [боевик палестинской экстремистской группировк...
10019    [11 сентября года день совершения, сентября го...
Name: shingles, Length: 10020, dtype: object

In [ ]:
df['shingles_len'] = df['shingles'].apply(len)
df['shingles_len'].max()

6088

In [ ]:
import random

def choose_shingles(shingles):
    shingles_count = 84 if len(shingles) > 84 else len(shingles)
    return random.choices(shingles, k=shingles_count)

choose_shingles(df['shingles'][0])

["нести министры '' сказал журналистам",
 'министры смогут самостоятельно соответствующих решений',
 'решений правительства создавать межведомственные комиссии',
 'сообщил министры смогут самостоятельно соответствующих',
 'работал сообщает риа `` новости',
 'предлагаемые решения должны нести министры',
 'министерствами `` уровень согласования заменит',
 'правительству упразднить значительную часть комиссий',
 'предлагаемые решения должны нести министры',
 'создавать межведомственные комиссии координации работы',
 'смогут самостоятельно соответствующих решений правительства',
 'именно закреплены задачи кудрин также',
 'таможенно-тарифной политике министерству экономического развития',
 'сообщил министры смогут самостоятельно соответствующих',
 'рф алексей кудрин предложил правительству',
 'часть комиссий которых работал сообщает',
 "нести министры '' сказал журналистам",
 'закреплены задачи кудрин также сообщил',
 'комиссии координации работы министерствами ``',
 'решений правительства 

In [ ]:
import fuzzywuzzy.fuzz

TRASHHOLD = 60

def equals(a, b):
    ratio = fuzzywuzzy.fuzz.token_sort_ratio(a, b)
    return ratio > TRASHHOLD
    
import time
s = time.time()
a = df['text'][10015]
b = df['text'][10019]
equals(a, b)


/Users/sultan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


False

In [ ]:
df['to_compare'] = df['shingles'].apply(choose_shingles)
df['to_compare']

0        [решений правительства создавать межведомствен...
1        [решение министра объясняется новыми требовани...
2        [машину сообщает ар это произошло, дороге маши...
3        [матки крохотные миоматозные узелки которые, п...
4        [протолкнет гной ткани углубит воспаление, ран...
                               ...                        
10015    [премьер-министр пакистана зафарулла хан джама...
10016    [карачи арестован член запрещенной исламистско...
10017    [исламистской организации которого обнаружена ...
10018    ['' четверг израильский военный суд, '' алам х...
10019    [отказывалась это делать основании законодател...
Name: to_compare, Length: 10020, dtype: object

In [ ]:
clusters = []
duplicates_count = 0
duplicates = set()
for i in range(1000):
    for j in range(i + 1, 1001):
        a = df['text'][i]
        b = df['text'][j]
        are_equal = equals(a, b)

        if are_equal:
            duplicates.add(j)

            cluster_found = False
            for cluster in clusters:
                if i in cluster or j in cluster:
                    cluster.add(i)
                    cluster.add(j)
                    cluster_found = True
            if not cluster_found:
                print(i, j)
                clusters.append({i, j})


778 888
974 975


In [ ]:
duplicates

{888, 975}

In [ ]:
clusters

[{778, 888}, {974, 975}]

In [ ]:
df.query('"" in shingles')

,docID,docURL,subject,agency,timestamp,content,text,shingles,shingles_len,to_compare


In [ ]:
clusters[0].issuperset(clusters[1])


False

In [ ]:
df['text'][778]

"екатеринбург апреля водитель направил микроавтобус дерево результате пострадали человека сообщили апи пресс-службе отдела гибдд увд екатеринбурга среду минут первого ночи улице бардина дома №33 водитель микроавтобуса `` тойота '' справился управлением наехал дерево обочине.-летний водитель получил черепно-мозговую травму его-летний пассажир получил перелом голени пострадавшие госпитализированы бригадой `` скорой помощи '' оба находились нетрезвом состоянии"

In [ ]:
df['text'][888]

"екатеринбург апреля результате наезда автомобиля столб екатеринбурге пострадали человека сообщили апи пресс-службе городского отдела гибдд минувшей ночью часа улице марта дома №163-летний водитель автомобиля ваз справился управлением из-за автомобиль врезался столб водитель пассажир машины госпитализированы места дтп бригадой `` скорой помощи '' городскую клиническую больницу №24 закрытыми черепно-мозговыми травмами"

In [ ]:
df['text'][974]

"міжнародний ярмарок багдаді перенесено пізніший термін зв'язку з убивством чотирьох американських цивільних фалуджі ярмарок мав відкритися наступного понеділка понад компаній з країн світу мали б представити свої товари однак американські офіційні представники заявили що зможуть гарантувати безпеки учасникам ярмарку повідомляє ``"

In [ ]:
df['text'][975]

'международная ярмарка багдаде перенесена поздний срок связи убийством четырех американских гражданских фалудже ярмарка должна открыться следующий понедельник компаний стран мира должны представить свои товары однако официальные американские представители заявили смогут гарантировать безопасности участникам ярмарки сообщает ``'